In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/vertex_endpoints/optimized_tensorflow_runtime/bert_optimized_online_prediction.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/vertex_endpoints/optimized_tensorflow_runtime/bert_optimized_online_prediction.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/vertex_endpoints/optimized_tensorflow_runtime/bert_optimized_online_prediction.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在 Vertex AI Workbench 中打开
    </a>
  </td>
</table>

# 调整 BERT 基础分类模型并将其部署到使用优化 TensorFlow 运行时的 Vertex AI 预测

## 概述

在这个示例中，您将学习如何微调BERT基础分类模型进行情感分析。

然后，您将使用基于开源的TensorFlow 2.7容器和优化的TensorFlow运行时容器将训练好的模型导出到Vertex AI预测服务，并对这些模型进行性能评估并比较它们的预测结果。

有关Vertex AI预测优化的TensorFlow运行时容器的其他信息，请参阅https://cloud.google.com/vertex-ai/docs/predictions/optimized-tensorflow-runtime。

### 数据集

这个笔记本训练一个情感分析模型，根据影评的文本将影评分类为积极或消极。

您将使用包含来自互联网电影数据库的50,000部影评的[大型影评数据集](https://ai.stanford.edu/~amaas/data/sentiment/)。

### 目标

在这个笔记本中，您将学习如何使用优化的TensorFlow运行时将微调的BERT分类模型部署到Vertex AI预测中。接下来，您将比较其性能与基于开源的TensorFlow容器的性能。

您执行的步骤包括：
* 下载并预处理[大型影评数据集](https://ai.stanford.edu/~amaas/data/sentiment/)
* 从TF hub下载BERT基础模型
* 微调BERT分类模型
* 使用TensorFlow 2.7容器将模型部署到Vertex AI预测
* 使用优化的TensorFlow运行时容器将模型部署到Vertex AI预测
* 对模型进行基准测试并验证它们的预测结果

您可以在Colab上微调BERT模型并将其上传到Vertex AI预测。为了获得可靠的基准测试结果，这个演示必须在与您的模型相同地区运行的Jupyter VM上运行。

### 成本

本教程使用Google Cloud的以下收费部分：

* Vertex AI
* 云存储

了解[Vertex AI的定价](https://cloud.google.com/vertex-ai/pricing)和[云存储的定价](https://cloud.google.com/storage/pricing)，并使用[定价计算器](https://cloud.google.com/products/calculator/)根据您的预计使用量生成成本估算。

### 设置您的本地开发环境

**如果您正在使用Colab或Vertex AI Workbench笔记本**，您的环境已经满足运行此笔记本的所有要求。您可以跳过这一步。

否则，请确保您的环境符合此笔记本的要求。
您需要以下内容：

* Google Cloud SDK
* Git
* Python 3
* virtualenv
* 在使用Python 3的虚拟环境中运行的Jupyter笔记本
* 安装了GPU驱动程序和CUDA 11.2

Google Cloud指南中的[设置Python开发环境](https://cloud.google.com/python/setup)和[Jupyter安装指南](https://jupyter.org/install)提供了详细的说明来满足这些要求。以下步骤提供了一套简洁的说明：

1. [安装并初始化Cloud SDK。](https://cloud.google.com/sdk/docs/)

2. [安装Python 3。](https://cloud.google.com/python/setup#installing_python)

3. [安装virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)并创建一个使用Python 3的虚拟环境。激活虚拟环境。

4. 要安装Jupyter，请在终端窗口中运行 `pip3 install jupyter`。

5. 要启动Jupyter，请在终端窗口中运行 `jupyter notebook`。

6. 在Jupyter Notebook Dashboard中打开此笔记本。

### 安装额外的包

安装在您的笔记本环境中尚未安装的额外包依赖项，如tensorflow、tensorflow-text、tensorflow serving APIs 和 Vertex AI SDK。请使用每个包的最新稳定版。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# Vertex AI Workbench Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_GOOGLE_CLOUD_NOTEBOOK:
    USER_FLAG = "--user"

In [ ]:
! pip3 install {USER_FLAG} --upgrade tensorflow==2.7.0 -q
! pip3 install {USER_FLAG} --upgrade tensorflow-text==2.7.0 -q
! pip3 install {USER_FLAG} --upgrade tensorflow-serving-api==2.7.0 -q
! pip3 install {USER_FLAG} --upgrade tf-models-official==2.7.0 -q
! pip3 install {USER_FLAG} --upgrade google-cloud-aiplatform -q
! pip3 install {USER_FLAG} --upgrade google-cloud-storage -q

### 重新启动内核

在安装额外的包之后，您必须重新启动笔记本内核，以便它可以找到这些包。

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

在开始之前

### 选择 GPU 运行时

**如果可以的话，请确保使用 GPU 运行时来运行这个笔记本。在 Colab 中，选择 "运行时 --> 更改运行时类型 > GPU"**。

请注意，为了能够使用 GPU 对模型进行微调，您的虚拟机需要安装 GPU 驱动程序和 CUDA 11.2。您可以使用 [TensorFlow Enterprise 2.7](https://cloud.google.com/tensorflow-enterprise/docs/use-with-notebooks) 用户管理的笔记本实例或在带有 GPU 运行时的 Colab 上进行操作。

### 设置您的Google Cloud项目

**以下步骤适用于所有笔记本环境。**

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建帐户时，您将获得$300的信用额度，用于支付计算和存储成本。

1. [确保您的项目已启用计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

1. [启用Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。

1. 如果您在本地运行此笔记本，您必须安装[Cloud SDK](https://cloud.google.com/sdk)。

1. 在以下单元格中输入您的项目ID。然后运行该单元格，以确保Cloud SDK为本笔记本中的所有命令使用正确的项目。

**注意**：Jupyter会运行以`!`为前缀的行作为shell命令，并将以`$`为前缀的Python变量插入这些命令中。

设置您的项目ID

**如果您不知道您的项目ID**，您可以尝试使用 `gcloud` 获取您的项目ID。

In [ ]:
import os

PROJECT_ID = ""

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

否则，请在这里设定您的项目ID。

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

时间戳

如果您在一个直播教程会话中，可能会使用一个共享的测试账户或项目。为了避免资源名称冲突，为每个实例会话创建一个时间戳，然后将其附加到您在本教程中创建的资源名称中。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的 Google Cloud 账户

**如果您正在使用 Vertex AI Workbench 笔记本**，您的环境已经经过验证。请跳过此步骤。

如果您正在使用Colab，请运行下面的单元格，并按照提示进行oAuth身份验证。

**否则**，请按照以下步骤操作：

1. 在Cloud控制台中，转到[**创建服务帐号密钥**页面](https://console.cloud.google.com/apis/credentials/serviceaccountkey)。

2. 点击**创建服务帐号**。

3. 在**服务帐号名称**字段中输入一个名称，然后点击**创建**。

4. 在**授予此服务帐号对项目的访问权限**部分，点击**角色**下拉列表。在过滤框中输入“Vertex AI”，并选择**Vertex AI管理员**。在过滤框中输入“存储对象管理员”，并选择**存储对象管理员**。

5. 点击**创建**。一个包含您密钥的JSON文件将下载到本地环境中。

6. 在以下单元格中将您的服务帐号密钥的路径作为`GOOGLE_APPLICATION_CREDENTIALS`变量输入，然后运行该单元格。

In [ ]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# The Vertex AI Workbench Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# If on  Vertex AI Workbench Notebooks, then don't execute this code
if not IS_GOOGLE_CLOUD_NOTEBOOK:
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储存储桶

**所有笔记本环境都需要执行以下步骤。**

为了让Vertex AI Prediction能够为您的模型提供服务，必须首先将其上传到云存储存储桶中。

请在下方设置您的云存储存储桶的名称。它必须是所有云存储存储桶中唯一的。

您还可以更改`REGION`变量，该变量在接下来的笔记本中的操作中使用。我们建议您[选择Vertex AI服务可用的区域](https://cloud.google.com/vertex-ai/docs/general/locations#available_regions)。

In [ ]:
BUCKET_URI = "gs://[your-bucket-name]"  # @param {type:"string"}
REGION = "[your-region]"  # @param {type:"string"}

In [ ]:
if BUCKET_URI == "" or BUCKET_URI is None or BUCKET_URI == "gs://[your-bucket-name]":
    BUCKET_URI = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

if REGION == "[your-region]":
    REGION = "us-central1"

只有在您的存储桶不存在时才运行以下单元格以创建您的云存储桶。

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

您的云存储桶的最后一步是通过检查其内容来验证对云存储桶的访问权限。

In [ ]:
! gsutil ls -al $BUCKET_URI

导入库并定义常量

In [ ]:
import json
import os
import shutil
import subprocess
import sys

import numpy as np
import requests as r
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text  # noqa: F401
from official.nlp import optimization  # to create AdamW optimizer

r.packages.urllib3.disable_warnings()

logging = tf.get_logger()
logging.propagate = False
logging.setLevel("INFO")

In [ ]:
LOCAL_DIRECTORY = "~/bert_classification"  # @param {type:"string"}
LOCAL_DIRECTORY_FULL = os.path.expanduser(LOCAL_DIRECTORY)

下载数据集

从[互联网电影数据库](https://www.imdb.com/)下载[大型电影评论数据集](https://ai.stanford.edu/~amaas/data/sentiment/)。该数据集包含了5万条电影评论的文本。

In [ ]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file(
    "aclImdb_v1.tar.gz", url, untar=True, cache_dir=".", cache_subdir=""
)

dataset_dir = os.path.join(os.path.dirname(dataset), "aclImdb")

train_dir = os.path.join(dataset_dir, "train")

# remove unused folders to make it easier to load the data
remove_dir = os.path.join(train_dir, "unsup")
shutil.rmtree(remove_dir)

## 预处理数据集

IMDB数据集已经被分为训练集和测试集，但缺少一个验证集。要创建一个验证集，在下一个单元格中，可以使用训练数据的80:20拆分来使用`validation_split`参数。

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
batch_size = 32
seed = 42

raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=batch_size,
    validation_split=0.2,
    subset="training",
    seed=seed,
)

class_names = raw_train_ds.class_names
train_ds = raw_train_ds.cache().prefetch(buffer_size=AUTOTUNE)

val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=batch_size,
    validation_split=0.2,
    subset="validation",
    seed=seed,
)

val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)

test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

看一下几篇评论。

In [ ]:
for text_batch, label_batch in train_ds.take(1):
    for i in range(3):
        print(f"Review: {text_batch.numpy()[i]}")
        label = label_batch.numpy()[i]
        print(f"Label : {label} ({class_names[label]})")

定义BERT基础分类模型

作为我们模型的基础，您可以从TensorFlow Hub中获取未经大小写处理的BERT-Base模型:
https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3

In [ ]:
bert_model_name = "bert_en_uncased_L-12_H-768_A-12"
tfhub_handle_encoder = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3"

为了将文本输入输入到模型中，数据需要使用来自TensorFlow hub的对应BERT预处理器进行预处理。

In [ ]:
tfhub_handle_preprocess = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"

In [ ]:
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)

通过将BERT编码器模型的输出输入到dropout和稠密层中来定义一个分类模型。
了解有关[使用BERT对文本进行分类](https://www.tensorflow.org/text/tutorials/classify_text_with_bert)的更多信息。

In [ ]:
def build_classifier_model():
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name="text")
    preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name="preprocessing")
    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name="BERT_encoder")
    outputs = encoder(encoder_inputs)
    net = outputs["pooled_output"]
    net = tf.keras.layers.Dropout(0.1)(net)
    net = tf.keras.layers.Dense(1, activation=tf.sigmoid, name="classifier")(net)
    return tf.keras.Model(text_input, net)

In [ ]:
classifier_model = build_classifier_model()

In [ ]:
epochs = 3
steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1 * num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(
    init_lr=init_lr,
    num_train_steps=num_train_steps,
    num_warmup_steps=num_warmup_steps,
    optimizer_type="adamw",
)

In [ ]:
loss = tf.keras.losses.BinaryCrossentropy()
metrics = tf.metrics.BinaryAccuracy()

In [ ]:
classifier_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

微调模型需要一些时间。

In [ ]:
print(f"Training model with {tfhub_handle_encoder}")
history = classifier_model.fit(x=train_ds, validation_data=val_ds, epochs=epochs)

评估模型。预计验证损失约为0.43。

In [ ]:
loss, accuracy = classifier_model.evaluate(test_ds)

print(f"Loss: {loss}")
print(f"Accuracy: {accuracy}")

导出用于推断的模型。

In [ ]:
!mkdir -p $LOCAL_DIRECTORY_FULL

In [ ]:
classifier_model.save(LOCAL_DIRECTORY_FULL, include_optimizer=False)

检查模型签名以查看预测请求应具有哪些字段。

请注意，您可能会看到有关缺少'CaseFoldUTF8'操作的堆栈跟踪消息。这是一个关于`saved_model_cli`的已知问题，您可以忽略此消息。

In [ ]:
!saved_model_cli show --dir $LOCAL_DIRECTORY_FULL --all

生成预测请求

现在您可以生成请求发送到我们的模型进行推理。请求以JSON Lines格式生成，每行一个请求。

In [ ]:
!mkdir -p $LOCAL_DIRECTORY_FULL/requests

In [ ]:
def encode(text):
    rows = []
    for row in text.numpy().tolist():
        rows.append(row.decode("utf-8"))

    return {"text": rows}


def export_requests_jsonl(file_name, rows=2, batch_size=32):
    with tf.io.gfile.GFile(file_name, mode="w") as f:
        for text in test_ds.unbatch().batch(batch_size).take(rows):
            d = encode(text[0])
            f.write(json.dumps(d))
            f.write("\n")

In [ ]:
export_requests_jsonl(
    os.path.join(LOCAL_DIRECTORY_FULL, "requests", "requests_1_1.jsonl"),
    rows=1,
    batch_size=1,
)
export_requests_jsonl(
    os.path.join(LOCAL_DIRECTORY_FULL, "requests", "requests_1_32.jsonl"),
    rows=1,
    batch_size=32,
)
export_requests_jsonl(
    os.path.join(LOCAL_DIRECTORY_FULL, "requests", "requests_10_32.jsonl"),
    rows=10,
    batch_size=32,
)
export_requests_jsonl(
    os.path.join(LOCAL_DIRECTORY_FULL, "requests", "requests_100_32.jsonl"),
    rows=100,
    batch_size=32,
)
export_requests_jsonl(
    os.path.join(LOCAL_DIRECTORY_FULL, "requests", "requests_1000_32.jsonl"),
    rows=1000,
    batch_size=32,
)

In [ ]:
!cat $LOCAL_DIRECTORY_FULL/requests/requests_1_1.jsonl

## （可选）生成热身请求

TensorFlow运行时有一些懒加载的组件。延迟初始化可能会导致加载模型后发送的第一个请求的延迟非常高。这种延迟可能比单个推理请求的延迟高几个数量级。

有关SavedModel预热的更多信息，请参阅https://www.tensorflow.org/tfx/serving/saved_model_warmup。

对于使用优化的TensorFlow运行时的Vertex AI预测，当模型预编译时，每个新批处理大小的第一个请求具有较高的延迟。当`allow_precompilation`标志设置为true时，预编译将启用。

为了减少高延迟，需为运行时提供一个用于启动时加载的预热请求。
预热文件应包含您预计模型在生产中收到的各种批处理大小。

请注意，使用多个批处理大小的预热请求会增加每个节点启动的时间。

如果您希望模型接收多个批处理大小，可以使用一组`allowed_batch_sizes`来进行自动服务器端请求批处理。更多信息，请参阅https://www.tensorflow.org/tfx/serving/serving_config#batching_configuration。

要为在Vertex AI Prediction上运行的模型启用自动批处理，请将批处理配置放入与saved_model.pb同一GCS目录中的[config/batching_parameters_config](https://cloud.google.com/vertex-ai/docs/training/exporting-model-artifacts#enable_server-side_request_batching_for_tensorflow)文件中。

In [ ]:
!mkdir -p $LOCAL_DIRECTORY_FULL/assets.extra

In [ ]:
from tensorflow_serving.apis import predict_pb2, prediction_log_pb2


def build_grpc_request(
    row_dict, model_name="default", signature_name="serving_default"
):
    """Generate gRPC inference request with payload."""

    request = predict_pb2.PredictRequest()
    request.model_spec.name = model_name
    request.model_spec.signature_name = signature_name
    for key, value in row_dict.items():
        proto = tf.make_tensor_proto(value)
        request.inputs[key].CopyFrom(proto)
    return request


def export_warmup_file(
    request_files, export_path, model_name="default", signature_name="serving_default"
):
    with tf.io.TFRecordWriter(export_path) as writer:
        for request_file_path in request_files:
            with open(request_file_path) as f:
                row_dict = json.loads(f.readline())
                request = build_grpc_request(row_dict, model_name, signature_name)
            log = prediction_log_pb2.PredictionLog(
                predict_log=prediction_log_pb2.PredictLog(request=request)
            )
            writer.write(log.SerializeToString())


export_warmup_file(
    [
        os.path.join(LOCAL_DIRECTORY_FULL, "requests", "requests_1_1.jsonl"),
        os.path.join(LOCAL_DIRECTORY_FULL, "requests", "requests_1_32.jsonl"),
    ],
    os.path.join(LOCAL_DIRECTORY_FULL, "assets.extra", "tf_serving_warmup_requests"),
)

将模型部署到Vertex AI 预测

要部署模型到 Vertex AI 预测服务，您必须将其放置在一个 GCS 存储桶中。

In [ ]:
!gsutil rm -r $BUCKET_URI/*

In [ ]:
!gsutil cp -r $LOCAL_DIRECTORY_FULL/* $BUCKET_URI

将Vertex AI Python客户端库导入到您的笔记本环墺中。

In [ ]:
from google.cloud.aiplatform import gapic as aip

定义部署时要使用的节点类型。要了解有关 Vertex AI 预测选项的信息，请参见 [配置计算资源](https://cloud.google.com/vertex-ai/docs/predictions/configure-compute)。

In [ ]:
DEPLOY_COMPUTE = "n1-standard-16"
DEPLOY_GPU = aip.AcceleratorType.NVIDIA_TESLA_T4

AI 平台的Python客户端库充当客户端/服务器模型。

在此示例中，您将使用以下客户端：
- 用于管理模型的模型服务。
- 用于部署的端点服务。
- 用于提供服务的预测服务。

In [ ]:
API_ENDPOINT = f"{REGION}-aiplatform.googleapis.com"
PARENT = f"projects/{PROJECT_ID}/locations/{REGION}"

client_options = {"api_endpoint": API_ENDPOINT}
model_service_client = aip.ModelServiceClient(client_options=client_options)
endpoint_service_client = aip.EndpointServiceClient(client_options=client_options)
prediction_service_client = aip.PredictionServiceClient(client_options=client_options)

### 将模型上传到Vertex AI预测

请查看[model_service.upload_model](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform_v1.services.model_service.ModelServiceClient#google_cloud_aiplatform_v1_services_model_service_ModelServiceClient_upload_model)文档以获取详细信息。

`artifact_uri`参数应指向一个GCS路径，该路径是您模型中`saved_model.pb`文件所在的位置。

`image_uri`指定要使用的docker镜像。在这里，您将使用TF2.7 GPU和Vertex AI Prediction优化的TensorFlow运行时图像上传相同的模型。

In [ ]:
tf27_gpu_model_dict = {
    "display_name": "BERT Base TF2.7 GPU model",
    "artifact_uri": BUCKET_URI,
    "container_spec": {
        "image_uri": "us-docker.pkg.dev/vertex-ai/prediction/tf2-gpu.2-7:latest",
    },
}
tf27_gpu_model = (
    model_service_client.upload_model(parent=PARENT, model=tf27_gpu_model_dict)
    .result(timeout=180)
    .model
)
tf27_gpu_model

使用Vertex AI Prediction优化的TensorFlow运行时部署模型时，请使用`us-docker.pkg.dev/vertex-ai-restricted/prediction/tf_opt-gpu.nightly:latest`容器。

模型应用了两种优化选项。
- *allow_precompilation* - 开启模型预编译以提高性能。请注意，当带有新批量大小的第一个请求到达时，模型预编译将会发生，并且在预编译完成后才会发送该请求的响应。为了减少这种情况，可以指定一个预热文件（请参见本文档早期部分）。模型预编译适用于不同类型的模型，在大多数情况下对性能有积极影响。但是，在生产环境中启用之前，我们建议您在您的模型上尝试一下。
- *allow_precision_affecting_optimizations* - 启用影响精度的优化。在一些情况下，这可以使模型运行速度显著加快，但会略微损失一些模型预测的准确性。在使用此优化时，您应该评估对模型精度的影响。

有关可用优化的TensorFlow运行时容器和选项列表，请参阅https://cloud.google.com/vertex-ai/docs/predictions/optimized-tensorflow-runtime。

In [ ]:
tf_opt_gpu_model_dict = {
    "display_name": "BERT Base optimized TensorFlow runtime GPU model",
    "artifact_uri": BUCKET_URI,
    "container_spec": {
        "image_uri": "us-docker.pkg.dev/vertex-ai-restricted/prediction/tf_opt-gpu.nightly:latest",
        "args": [
            "--allow_precompilation=true",
            "--allow_precision_affecting_optimizations=false",
        ],
    },
}

tf_opt_gpu_model = (
    model_service_client.upload_model(parent=PARENT, model=tf_opt_gpu_model_dict)
    .result(timeout=180)
    .model
)
tf_opt_gpu_model

In [ ]:
tf_opt_lossy_gpu_model_dict = {
    "display_name": "BERT Base optimized TensorFlow runtime GPU model with lossy optimizations",
    "artifact_uri": BUCKET_URI,
    "container_spec": {
        "image_uri": "us-docker.pkg.dev/vertex-ai-restricted/prediction/tf_opt-gpu.nightly:latest",
        "args": [
            "--allow_precompilation=true",
            "--allow_precision_affecting_optimizations=true",
        ],
    },
}

tf_opt_lossy_gpu_model = (
    model_service_client.upload_model(parent=PARENT, model=tf_opt_lossy_gpu_model_dict)
    .result(timeout=180)
    .model
)
tf_opt_lossy_gpu_model

列出所有的型号。

In [ ]:
model_service_client.list_models(parent=PARENT)

创建端点

了解有关[endpoint_service.create_endpoint](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform_v1.services.endpoint_service.EndpointServiceClient#google_cloud_aiplatform_v1_services_endpoint_service_EndpointServiceClient_create_endpoint)的更多信息。

In [ ]:
tf27_gpu_endpoint_dict = {
    "display_name": "BERT Base TF2.7 GPU endpoint",
}
tf27_gpu_endpoint = (
    endpoint_service_client.create_endpoint(
        parent=PARENT, endpoint=tf27_gpu_endpoint_dict
    )
    .result(timeout=300)
    .name
)
tf27_gpu_endpoint

In [ ]:
tf_opt_gpu_endpoint_dict = {
    "display_name": "BERT Base optimized TensorFlow runtime GPU endpoint",
}
tf_opt_gpu_endpoint = (
    endpoint_service_client.create_endpoint(
        parent=PARENT, endpoint=tf_opt_gpu_endpoint_dict
    )
    .result(timeout=300)
    .name
)
tf_opt_gpu_endpoint

In [ ]:
tf_opt_lossy_gpu_endpoint_dict = {
    "display_name": "BERT Base optimized TensorFlow runtime GPU with lossy optimizations endpoint",
}
tf_opt_lossy_gpu_endpoint = (
    endpoint_service_client.create_endpoint(
        parent=PARENT, endpoint=tf_opt_lossy_gpu_endpoint_dict
    )
    .result(timeout=300)
    .name
)
tf_opt_lossy_gpu_endpoint

### 将模型部署到端点

了解有关[enpoint_service.deploy_model](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform_v1.services.endpoint_service.EndpointServiceClient#google_cloud_aiplatform_v1_services_endpoint_service_EndpointServiceClient_deploy_model)的更多信息。

In [ ]:
tf27_gpu_deployed_model_dict = {
    "model": tf27_gpu_model,
    "display_name": "BERT Base TF2.7 GPU deployed model",
    "dedicated_resources": {
        "min_replica_count": 1,
        "max_replica_count": 1,
        "machine_spec": {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_type": DEPLOY_GPU,
            "accelerator_count": 1,
        },
    },
}

tf27_gpu_deployed_model = endpoint_service_client.deploy_model(
    endpoint=tf27_gpu_endpoint,
    deployed_model=tf27_gpu_deployed_model_dict,
    traffic_split={"0": 100},
).result()
tf27_gpu_deployed_model

In [ ]:
tf_opt_gpu_deployed_model_dict = {
    "model": tf_opt_gpu_model,
    "display_name": "BERT Base optimized TensorFlow runtime GPU model",
    "dedicated_resources": {
        "min_replica_count": 1,
        "max_replica_count": 1,
        "machine_spec": {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_type": DEPLOY_GPU,
            "accelerator_count": 1,
        },
    },
}

tf_opt_gpu_deployed_model = endpoint_service_client.deploy_model(
    endpoint=tf_opt_gpu_endpoint,
    deployed_model=tf_opt_gpu_deployed_model_dict,
    traffic_split={"0": 100},
).result()
tf_opt_gpu_deployed_model

In [ ]:
tf_opt_lossy_gpu_deployed_model_dict = {
    "model": tf_opt_lossy_gpu_model,
    "display_name": "BERT Base optimized TensorFlow runtime GPU model with lossy optimizations",
    "dedicated_resources": {
        "min_replica_count": 1,
        "max_replica_count": 1,
        "machine_spec": {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_type": DEPLOY_GPU,
            "accelerator_count": 1,
        },
    },
}

tf_opt_lossy_gpu_deployed_model = endpoint_service_client.deploy_model(
    endpoint=tf_opt_lossy_gpu_endpoint,
    deployed_model=tf_opt_lossy_gpu_deployed_model_dict,
    traffic_split={"0": 100},
).result()
tf_opt_lossy_gpu_deployed_model

发送预测请求

现在您可以使用`prediction_service_client.predict` API向您的模型发送预测请求。

In [ ]:
prediction_service_client.predict(
    endpoint=tf27_gpu_endpoint,
    instances=["This was the best movie ever", "Movie was boring"],
)

或者，您可以在不使用SDK的情况下发送POST REST请求。了解更多信息，请访问https://cloud.google.com/vertex-ai/docs/predictions/online-predictions-custom-models#online_predict_custom_trained-drest。这种方法稍微更快。

In [ ]:
def get_headers():
    gcloud_access_token = (
        subprocess.check_output("gcloud auth print-access-token".split(" "))
        .decode()
        .rstrip("\n")
    )
    return {"authorization": "Bearer " + gcloud_access_token}


def send_post_request(uri, request_dict):
    return r.post(
        uri, data=json.dumps(request_dict), headers=get_headers(), verify=False
    )


uri = f"https://{REGION}-aiplatform.googleapis.com/v1/{tf27_gpu_endpoint}:predict"
print(uri)

request = {
    "instances": [
        {"text": "This was the best movie ever"},
        {"text": "Movie was boring"},
    ]
}
response = send_post_request(uri, request)

print(response.text)

##（可选） 部署模型进行基准测试

您可以在Colab环境中运行基准测试，为了获得可靠的结果，您应该使用与您的模型相同区域的虚拟机。

导入用于基准测试模型的帮助函数。

In [ ]:
!curl https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/vertex_endpoints/optimized_tensorflow_runtime/benchmark.py -o benchmark.py

In [ ]:
from benchmark import benchmark

该代码以给定的QPS异步且均匀地发送指定数量的请求，然后记录观察到的延迟。接下来，延迟结果被合并并计算百分位数。
模型可以处理的`actual_qps` 是通过模型处理发送的请求所需的时间除以请求数来计算的。
通过为`send_request` 和 `build_request` 函数提供不同的实现，可以使用相同的代码对在本地或在 Vertex AI Prediction 上使用 gRPC 和 REST 协议运行的模型进行基准测试。

这个基准测试的主要目标是在不同负载下测量模型延迟和模型可以处理的最大吞吐量。为了找到最大吞吐量，逐渐增加 QPS 直到`actual_qps` 不再增加而延迟急剧增加为止。

在生产部署中，工作负载并不是均匀的，因此最大模型吞吐量可能会较低。
我们并不打算在这里模拟生产工作负载。这个基准测试旨在比较在不同环境下运行相同模型的延迟和吞吐量。

In [ ]:
def build_rest_request(row_dict, model_name):
    payload = json.dumps({"instances": row_dict})
    return payload

In [ ]:
headers = get_headers()


def send_rest_request(request):
    res = r.post(
        f"https://{REGION}-aiplatform.googleapis.com/v1/{tf27_gpu_endpoint}:predict",
        data=request,
        headers=headers,
        verify=False,
    )
    assert res.status_code == 200
    return res


tf27_gpu_results = benchmark(
    send_rest_request,
    build_rest_request,
    f"{LOCAL_DIRECTORY_FULL}/requests/requests_10_32.jsonl",
    [1, 2, 3, 4, 5],
    5,
)

tf27_gpu_results

In [ ]:
headers = get_headers()


def send_rest_request(request):
    res = r.post(
        f"https://{REGION}-aiplatform.googleapis.com/v1/{tf_opt_gpu_endpoint}:predict",
        data=request,
        headers=headers,
        verify=False,
    )
    assert res.status_code == 200
    return res


tf_opt_gpu_results = benchmark(
    send_rest_request,
    build_rest_request,
    f"{LOCAL_DIRECTORY_FULL}/requests/requests_10_32.jsonl",
    [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    5,
)

tf_opt_gpu_results

In [ ]:
headers = get_headers()


def send_rest_request(request):
    res = r.post(
        f"https://{REGION}-aiplatform.googleapis.com/v1/{tf_opt_lossy_gpu_endpoint}:predict",
        data=request,
        headers=headers,
        verify=False,
    )
    assert res.status_code == 200
    return res


tf_opt_lossy_gpu_results = benchmark(
    send_rest_request,
    build_rest_request,
    f"{LOCAL_DIRECTORY_FULL}/requests/requests_10_32.jsonl",
    [1, 5, 10, 15, 20, 21, 22, 23, 24, 25],
    5,
)

tf_opt_lossy_gpu_results

合并和可视化结果。

In [ ]:
import matplotlib
import matplotlib.pyplot as plt


def build_graph(x_key, y_key, results_dict, axis):
    matplotlib.rcParams["figure.figsize"] = [10.0, 7.0]

    fig, ax = plt.subplots(facecolor=(1, 1, 1))
    ax.set_xlabel("QPS")
    ax.set_ylabel("Latency(ms)")
    for title, results in results_dict.items():
        x = np.array(results[x_key])
        y = np.array(results[y_key])
        ax.plot(x, y, label=title)
    ax.legend()
    ax.axis(axis)
    ax.set_title(f"BERT base model {y_key} latency, batch size 32")
    return fig

In [ ]:
fig = build_graph(
    "actual_qps",
    "p50",
    {
        "TF2.7 GPU": tf27_gpu_results,
        "TF opt GPU": tf_opt_gpu_results,
        "TF opt GPU lossy": tf_opt_lossy_gpu_results,
    },
    (0, 14, 0, 1000),
)
fig.savefig("bert_p50_latency_32.png", bbox_inches="tight")

In [ ]:
fig = build_graph(
    "actual_qps",
    "p99",
    {
        "TF2.7 GPU": tf27_gpu_results,
        "TF opt GPU": tf_opt_gpu_results,
        "TF opt GPU lossy": tf_opt_lossy_gpu_results,
    },
    (0, 14, 0, 1000),
)
fig.savefig("bert_p99_latency_32.png", bbox_inches="tight")

您可以看到，Vertex AI Prediction 优化的 TensorFlow 运行时与 TensorFlow 2.7 相比，具有明显更高的吞吐量和更低的延迟。

## （可选）使用MLPerf推理负载生成器比较部署模型的性能

MLPerf推理是一个基准套件，用于测量系统在各种部署场景中可以运行模型的速度。MLPerf现在是衡量模型性能的行业标准方式。您可以按照 https://github.com/tensorflow/tpu/tree/master/models/experimental/inference/load_test 上的说明来运行已部署模型的MLPerf推理基准测试。

（可选）比较预测结果

在这个示例中，使用了优化的TensorFlow运行时进行顶点预测，并将`allow_precision_affecting_optimizations`标志设置为`true`，以获得额外的加速。现在让我们检查这些优化如何影响预测结果。

比较在优化的TensorFlow运行时和TF2.7上运行的模型对32,000个请求的预测结果。

In [ ]:
def get_predictions(endpoint, requests_file_path):
    responses = []

    with tf.io.gfile.GFile(requests_file_path, "r") as f:
        for line in f:
            row_dict = json.loads(line)
            response = prediction_service_client.predict(
                endpoint=endpoint,
                instances=row_dict["text"],
            )
            for prediction in response.predictions:
                responses.append(prediction[0])

    return np.array(responses)

In [ ]:
tf27_gpu_predictions = get_predictions(
    tf27_gpu_endpoint, f"{LOCAL_DIRECTORY_FULL}/requests/requests_1000_32.jsonl"
)

In [ ]:
tf_opt_lossy_gpu_predictions = get_predictions(
    tf_opt_lossy_gpu_endpoint, f"{LOCAL_DIRECTORY_FULL}/requests/requests_1000_32.jsonl"
)

In [ ]:
np.average(tf_opt_lossy_gpu_predictions - tf27_gpu_predictions) * 100

In [ ]:
np.max(np.abs(tf_opt_lossy_gpu_predictions - tf27_gpu_predictions)) * 100

你可以看到，平均结果在少于0.01%时是不同的。在最坏的情况下，差异小于1%。

清理

完成后，可以安全地移除您创建的端点和部署的模型。

In [ ]:
def cleanup(endpoint, model_name, deployed_model_id):
    response = endpoint_service_client.undeploy_model(
        endpoint=endpoint, deployed_model_id=deployed_model_id
    )
    print("running undeploy_model operation:", response.operation.name)
    print(response.result())

    response = endpoint_service_client.delete_endpoint(name=endpoint)
    print("running delete_endpoint operation:", response.operation.name)
    print(response.result())

    response = model_service_client.delete_model(name=model_name)
    print("running delete_model operation:", response.operation.name)
    print(response.result())

In [ ]:
cleanup(tf27_gpu_endpoint, tf27_gpu_model, tf27_gpu_deployed_model.deployed_model.id)
cleanup(
    tf_opt_gpu_endpoint, tf_opt_gpu_model, tf_opt_gpu_deployed_model.deployed_model.id
)
cleanup(
    tf_opt_lossy_gpu_endpoint,
    tf_opt_lossy_gpu_model,
    tf_opt_lossy_gpu_deployed_model.deployed_model.id,
)

你现在也可以从GCS存储桶中删除模型。

In [ ]:
# Set this to true only if you'd like to delete your bucket
delete_bucket = False
if delete_bucket or os.getenv("IS_TESTING"):
    !gsutil rm -r $BUCKET_URI